In [1]:
import numpy as np 
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split
import xgboost as xgb
import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/lib/kaggle/competitions/twosigmanews/__init__.py
/kaggle/lib/kaggle/competitions/twosigmanews/env.py
/kaggle/input/taxi-fare-pre/__results__.html
/kaggle/input/taxi-fare-pre/test_prep.csv
/kaggle/input/taxi-fare-pre/custom.css
/kaggle/input/taxi-fare-pre/train_prep10M.feather
/kaggle/input/taxi-fare-pre/__notebook__.ipynb
/kaggle/input/taxi-fare-pre/__output__.json
/kaggle/input/taxi-fare-pre/__results___files/__results___31_0.png
/kaggle/input/taxi-fare-pre/__results___files/__results___20_0.png
/kaggle/input/taxi-fare-pre/__results___files/__results___16_0.png
/kaggle/input/taxi-fare-pre/__results___files/__results___28_0.png
/kaggle/input/taxi-fare-pre/__results___files/__results___26_0.png
/kaggle/input/taxi-fare-pre/__results___files/__results___37_0.png
/kaggle/input/taxi-fare-pre/__results___files/__results___33_0.png
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-predict

Lendo o dataset de treino

In [2]:
train_df = pd.read_feather('/kaggle/input/taxi-fare-pre/train_prep10M.feather')
train_df.drop(columns=['index'], inplace=True)
train_df.dtypes

fare_amount          float32
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count        uint8
year                   int64
month                  int64
day                    int64
hour                   int64
weekday                int64
haversine            float64
dtype: object

# Separando os conjuntos de treino e validação

In [1]:
train_df, validation_df = train_test_split(train_df, test_size=0.10, random_state=1)
X_train, y_train = train_df.values[:, 1:], train_df.values[:, 0]
X_validation, y_validation = validation_df.values[:, 1:], validation_df.values[:, 0]

NameError: name 'train_test_split' is not defined

In [4]:
#Cross-validation
params = {
    # Parametros que vão ser ajustados
    'max_depth': 8, 
    'eta':.03,
    'subsample': 1,
    'colsample_bytree': 0.8,
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 1
}
print(params)

{'max_depth': 8, 'eta': 0.03, 'subsample': 1, 'colsample_bytree': 0.8, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': 0}


In [5]:
def XGBmodel(X_train,X_validation,y_train,y_validation,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_validation,label=y_validation)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=50, 
                    early_stopping_rounds=5,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_validation,y_train,y_validation,params)

[18:38:14] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[18:38:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 478 extra nodes, 0 pruned nodes, max_depth=8
[0]	test-rmse:13.7062
Will train until test-rmse hasn't improved in 5 rounds.
[18:39:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 504 extra nodes, 0 pruned nodes, max_depth=8
[1]	test-rmse:13.3277
[18:39:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 478 extra nodes, 0 pruned nodes, max_depth=8
[2]	test-rmse:12.9604
[18:39:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 486 extra nodes, 0 pruned nodes, max_depth=8
[3]	test-rmse:12.621
[18:40:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 502 extra nodes, 0 pruned nodes, max_depth=8
[4]	test-rmse:12.2746
[18:40:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 470 extra nodes, 0 pruned node

# Predição

In [6]:
# Lendo o dataset de teste
test_df =  pd.read_csv('/kaggle/input/taxi-fare-pre/test_prep.csv')

test_key = test_df['key']
test_values = test_df.drop(columns=['key', 'Unnamed: 0']).values
test_df.head()

,Unnamed: 0,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,weekday,haversine
0,0,2015-01-27 13:08:24.0000002,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13,1,0.021542
1,1,2015-01-27 13:08:24.0000003,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13,1,0.023141
2,2,2011-10-08 11:53:44.0000002,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11,5,0.005864
3,3,2012-12-01 21:12:12.0000002,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21,5,0.018634
4,4,2012-12-01 21:12:12.0000003,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21,5,0.050604


In [7]:
# Predizendo dos dados de teste
prediction = model.predict(xgb.DMatrix(test_values), ntree_limit = model.best_ntree_limit)
prediction

array([ 7.707834 ,  8.068994 ,  4.2174015, ..., 41.754143 , 15.802537 ,
        5.9898705], dtype=float32)

In [8]:
#Fazendo o aquivo para ser submetido no kaggle
submission = pd.DataFrame({
        "key": test_key,
        "fare_amount": prediction.round(2)
})

submission.to_csv('taxi_fare_submission.csv',index=False)
submission.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,7.71
1,2015-01-27 13:08:24.0000003,8.07
2,2011-10-08 11:53:44.0000002,4.22
3,2012-12-01 21:12:12.0000002,6.27
4,2012-12-01 21:12:12.0000003,11.23


O resultado obtido foi aproximadamente 4.75 como RMSE